In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

cmi_detect_behavior_with_sensor_data_path = kagglehub.competition_download('cmi-detect-behavior-with-sensor-data')

print('Data source import complete.')

In [ ]:
!pip install catboost -qq


In [ ]:
!pip install iterative-stratification==0.1.7 -qq

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import sklearn
import joblib
import warnings

pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)
pd.set_option('future.no_silent_downcasting', True)
warnings.filterwarnings('ignore')

In [ ]:
print(pd.__version__)
print(np.__version__)
print(pl.__version__)
print(sklearn.__version__)
print(joblib.__version__)

In [ ]:
%%time

train = pl.read_csv(f'{cmi_detect_behavior_with_sensor_data_path}/train.csv')
test = pl.read_csv(f'{cmi_detect_behavior_with_sensor_data_path}/test.csv')
train_demographics = pl.read_csv(f'{cmi_detect_behavior_with_sensor_data_path}/train_demographics.csv')
test_demographics = pl.read_csv(f'{cmi_detect_behavior_with_sensor_data_path}/test_demographics.csv')

train = train.to_pandas()
test = test.to_pandas()
train_demographics = train_demographics.to_pandas()
test_demographics = test_demographics.to_pandas()

train = pd.merge(train, train_demographics, on='subject', how='left')
test = pd.merge(test, test_demographics, on='subject', how='left')

train.head(1)

In [ ]:
train.gesture.unique()

# EDA

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# for target in train['gesture'].unique():
#     print(f"--- Analyzing gesture: {target} ---")

#     gesture_df = train[train['gesture'] == target].copy()
#     print(f"Found {gesture_df['sequence_id'].nunique()} sequences.")

#     aligned_dfs = []
#     for seq_id in gesture_df['sequence_id'].unique():
#         seq_data = gesture_df[gesture_df['sequence_id'] == seq_id].copy()
#         try:
#             gesture_start_time = seq_data[seq_data['behavior'] == 'Performs gesture']['sequence_counter'].iloc[0]
#             seq_data['relative_time'] = seq_data['sequence_counter'] - gesture_start_time
#             aligned_dfs.append(seq_data)
#         except IndexError:
#             continue

#     if not aligned_dfs:
#         print("No valid sequences with a 'Gesture' phase found. Skipping plot.\n")
#         continue

#     aligned_df = pd.concat(aligned_dfs)

#     if len(aligned_df['handedness'].unique()) < 2:
#         print("Not enough handedness diversity for this gesture. Skipping plot.\n")
#         continue

#     stats_df = aligned_df.groupby(['handedness', 'relative_time']).agg(
#         acc_x_mean=('acc_x', 'mean'), acc_x_std=('acc_x', 'std'),
#         acc_y_mean=('acc_y', 'mean'), acc_y_std=('acc_y', 'std'),
#         acc_z_mean=('acc_z', 'mean'), acc_z_std=('acc_z', 'std'),
#         rot_w_mean=('rot_w', 'mean'), rot_w_std=('rot_w', 'std'),
#         rot_x_mean=('rot_x', 'mean'), rot_x_std=('rot_x', 'std'),
#         rot_y_mean=('rot_y', 'mean'), rot_y_std=('rot_y', 'std'),
#         rot_z_mean=('rot_z', 'mean'), rot_z_std=('rot_z', 'std'),
#     ).reset_index()

#     rh_stats = stats_df[stats_df['handedness'] == 1]
#     lh_stats = stats_df[stats_df['handedness'] == 0]

#     # --- 4a. Plot ACCELEROMETER Data ---
#     fig_acc, axes_acc = plt.subplots(3, 1, figsize=(18, 15), sharex=True)
#     fig_acc.suptitle(f'Mean Accelerometer Signals for "{target}" (Aligned at Gesture Start)', fontsize=18)
#     axes_acc[0].set_xlim(-30, 60)

#     for i, axis in enumerate(['x', 'y', 'z']):
#         ax = axes_acc[i]
#         ax.plot(rh_stats['relative_time'], rh_stats[f'acc_{axis}_mean'], color='blue', linewidth=2, label='Mean Right-Handed')
#         ax.fill_between(rh_stats['relative_time'], rh_stats[f'acc_{axis}_mean'] - rh_stats[f'acc_{axis}_std'], rh_stats[f'acc_{axis}_mean'] + rh_stats[f'acc_{axis}_std'], color='blue', alpha=0.1)
#         ax.plot(lh_stats['relative_time'], -lh_stats[f'acc_{axis}_mean'], color='darkred', linestyle='--', linewidth=2, label='Mean -1 * Left-Handed (Hypothesis)')
#         ax.fill_between(lh_stats['relative_time'], -lh_stats[f'acc_{axis}_mean'] - lh_stats[f'acc_{axis}_std'], -lh_stats[f'acc_{axis}_mean'] + lh_stats[f'acc_{axis}_std'], color='red', alpha=0.1)
#         ax.axvline(x=0, color='black', linestyle=':', linewidth=3, label='Gesture Start (t=0)')
#         ax.set_ylabel(f'acc_{axis}', fontsize=12)
#         ax.grid(True, which='both', linestyle='--')
#         ax.legend()
#     plt.xlabel('Time Relative to Gesture Start (steps)', fontsize=14)
#     plt.show()

#     # --- 4b. Plot ROTATION Data ---
#     fig_rot, axes_rot = plt.subplots(4, 1, figsize=(18, 20), sharex=True)
#     fig_rot.suptitle(f'Mean Rotation (Quaternion) Signals for "{target}" (Aligned at Gesture Start)', fontsize=18)
#     axes_rot[0].set_xlim(-30, 60)

#     for i, axis in enumerate(['w', 'x', 'y', 'z']):
#         ax = axes_rot[i]
#         ax.plot(rh_stats['relative_time'], rh_stats[f'rot_{axis}_mean'], color='blue', linewidth=2, label='Mean Right-Handed')
#         ax.fill_between(rh_stats['relative_time'], rh_stats[f'rot_{axis}_mean'] - rh_stats[f'rot_{axis}_std'], rh_stats[f'rot_{axis}_mean'] + rh_stats[f'rot_{axis}_std'], color='blue', alpha=0.1)
#         ax.plot(lh_stats['relative_time'], -lh_stats[f'rot_{axis}_mean'], color='darkred', linestyle='--', linewidth=2, label='Mean -1 * Left-Handed (Hypothesis)')
#         ax.fill_between(lh_stats['relative_time'], -lh_stats[f'rot_{axis}_mean'] - lh_stats[f'rot_{axis}_std'], -lh_stats[f'rot_{axis}_mean'] + lh_stats[f'rot_{axis}_std'], color='red', alpha=0.1)
#         ax.axvline(x=0, color='black', linestyle=':', linewidth=3, label='Gesture Start (t=0)')
#         ax.set_ylabel(f'rot_{axis}', fontsize=12)
#         ax.grid(True, which='both', linestyle='--')
#         ax.legend()
#     plt.xlabel('Time Relative to Gesture Start (steps)', fontsize=14)
#     plt.show()

#     print("-" * 40 + "\n")

# CONFIG

In [ ]:
class CONFIG:
  TARGET = "gesture"
  SUBJECT = "subject"
  TRAIN_ONLY_COLS = ['sequence_type', 'subject', 'orientation', 'behavior', 'phase', 'gesture']
  NUM_CLASSES = train.gesture.nunique()
  FOLDS = 5
  ERR = 1e-8

imu_cols = [
            "acc_x", "acc_y", "acc_z",
            "rot_w", "rot_x", "rot_y", "rot_z",
            "acc_mag",

            "euler_roll", "euler_pitch", "euler_yaw",
            "euler_total", "pitch_roll_ratio", "yaw_pitch_ratio",

            "rot_matrix_r11", "rot_matrix_r12", "rot_matrix_r13",
            "rot_matrix_r21", "rot_matrix_r22", "rot_matrix_r23",
            "rot_matrix_r31", "rot_matrix_r32", "rot_matrix_r33",

            "angular_jerk_x", "angular_jerk_y", "angular_jerk_z",
            ]


demo_cols = ["adult_child", "age", "sex", "handedness", "height_cm", "shoulder_to_wrist_cm", "elbow_to_wrist_cm"]

thm_cols = ["thm_1", "thm_2","thm_3","thm_4","thm_5"]
tof_cols = [col for col in test.columns if col.startswith("tof_")]

seq_agg_cols = []
for col in imu_cols:
    seq_agg_cols.extend([
        f'{col}_seq_mean',
        f'{col}_seq_std',
        f'{col}_seq_min',
        ])


position_cols = []
for col in imu_cols:
    position_cols.extend([

        f'{col}_avg_velocity',

        f'{col}_early_mean',
        f'{col}_mid_mean',
        f'{col}_mid2_mean',
        f'{col}_mid3_mean',
        f'{col}_late_mean',

        f'{col}_early_std',
        f'{col}_mid_std',
        f'{col}_mid2_std',
        f'{col}_mid3_std',
        f'{col}_late_std',

        f'{col}_early_velocity_mean',
        f'{col}_mid_velocity_mean',
        f'{col}_mid2_velocity_mean',
        f'{col}_mid3_velocity_mean',
        f'{col}_late_velocity_mean',

        f'{col}_early_velocity_std',
        f'{col}_mid_velocity_std',
        f'{col}_mid2_velocity_std',
        f'{col}_mid3_velocity_std',
        f'{col}_late_velocity_std',

        # f'{col}_early_energy',
        # f'{col}_mid_energy',
        # f'{col}_mid2_energy',
        # f'{col}_mid3_energy',
        f'{col}_late_energy',

        f'{col}_very_late_mean',
        f'{col}_very_late_std',

        f'{col}_early_late_mean_ratio',
        f'{col}_early_late_std_ratio',
        f'{col}_early_late_energy_ratio',

    ])

for col in imu_cols:
    position_cols.extend([
        f"{col}_late_gesture_zero_crossing_rate_std",
        # f"{col}_dwell_time",
    ])

corr_cols = []
axis_pairs = [('acc_x', 'acc_y'), ('acc_x', 'acc_z'), ('acc_y', 'acc_z')]
for axis1, axis2 in axis_pairs:
    corr_cols.extend([
        f'{axis1}_{axis2}_corr',
    ])

euler_pairs = [('euler_roll', 'euler_pitch'), ('euler_roll', 'euler_yaw'), ('euler_pitch', 'euler_yaw')]
for axis1, axis2 in euler_pairs:
    corr_cols.extend([
        f'{axis1}_{axis2}_corr',
    ])


# FEATURES = imu_cols + seq_agg_cols + temporal_cols + position_cols + corr_cols
FEATURES = imu_cols + seq_agg_cols + position_cols + corr_cols


In [ ]:
len(FEATURES), len(imu_cols)

In [ ]:
train.shape

In [ ]:
train.phase.unique()

In [ ]:
train.sequence_type.unique()

## FEATURE ENGINEERING

In [ ]:
train.head(1)

In [ ]:
train = train[train['subject'] != 'SUBJ_011323']
# # # train = train[train['subject'] != 'SUBJ_045235']
# # # train = train[train['subject'] != 'SUBJ_019262']
train = train[train['sequence_id'] != 'SEQ_011975']

# # for seq_id in missing_rot_sequences:
# #   train = train[train['sequence_id'] != seq_id]


In [ ]:
def noise_augmentation(df):
    """Add realistic sensor noise"""
    print(f"Original shape: {df.shape}")
    print("Adding noise")
    # augmented_df = df[df["sequence_type"] == "Target"].copy()
    augmented_df = df.copy()

    # Add small amounts of Gaussian noise (realistic for sensor data)
    for col in ['acc_x', 'acc_y', 'acc_z']:
        noise = np.random.normal(0, 0.1, len(augmented_df))
        augmented_df[col] += noise

    augmented_df["sequence_id"] = augmented_df["sequence_id"].astype(str) + "_noise"

    final_df = pd.concat([df, augmented_df], ignore_index=True)
    print(f"Final shape: {final_df.shape}")

    return final_df


In [ ]:
def cast_to_object(df):
  df['adult_child'] = df['adult_child'].astype("category")
  df['sex'] = df['sex'].astype("category")
  df['handedness'] = df['handedness'].astype("category")
  return df


def aggregation_features(df):
    df = pl.from_pandas(df)
    agg_exprs = []

    for col in imu_cols:
        agg_exprs.extend([
            # Basic statistics
            pl.col(col).mean().over('sequence_id').alias(f'{col}_seq_mean'),
            pl.col(col).std().over('sequence_id').alias(f'{col}_seq_std'),
            pl.col(col).min().over('sequence_id').alias(f'{col}_seq_min'),
        ])

    df = df.with_columns(agg_exprs)
    return df.to_pandas()

def quaternion_to_euler(w, x, y, z):
    """Convert quaternion to Euler angles (roll, pitch, yaw) in radians"""

    sinr_cosp = 2 * (w * x + y * z)
    cosr_cosp = 1 - 2 * (x * x + y * y)
    roll = np.arctan2(sinr_cosp, cosr_cosp)

    sinp = 2 * (w * y - z * x)
    pitch = np.where(np.abs(sinp) >= 1, np.copysign(np.pi / 2, sinp), np.arcsin(sinp))

    siny_cosp = 2 * (w * z + x * y)
    cosy_cosp = 1 - 2 * (y * y + z * z)
    yaw = np.arctan2(siny_cosp, cosy_cosp)

    return roll, pitch, yaw

def mag_features(df):

    df["acc_mag"] = np.sqrt(df["acc_x"]**2 + df["acc_y"]**2 + df["acc_z"]**2)
    df["rot_mag"] = np.sqrt(df["rot_x"]**2 + df["rot_y"]**2 + df["rot_z"]**2)

    roll, pitch, yaw = quaternion_to_euler(df["rot_w"], df["rot_x"], df["rot_y"], df["rot_z"])

    df["euler_roll"] = roll
    df["euler_pitch"] = pitch
    df["euler_yaw"] = yaw

    # Euler angle magnitudes and combinations
    df["euler_total"] = np.sqrt(roll**2 + pitch**2 + yaw**2)
    df["pitch_roll_ratio"] = pitch / (np.abs(roll) + CONFIG.ERR)
    df["yaw_pitch_ratio"] = yaw / (np.abs(pitch) + CONFIG.ERR)

    return df

def rotation_matrix_features(df):
    """Extract features from rotation matrices - captures 3D orientation relationships"""
    df = pl.from_pandas(df)


    rot_matrix_exprs = []


    rot_matrix_exprs.extend([
        (1 - 2*(pl.col('rot_y')**2 + pl.col('rot_z')**2)).alias('rot_matrix_r11'),
        (2*(pl.col('rot_x')*pl.col('rot_y') - pl.col('rot_w')*pl.col('rot_z'))).alias('rot_matrix_r12'),
        (2*(pl.col('rot_x')*pl.col('rot_z') + pl.col('rot_w')*pl.col('rot_y'))).alias('rot_matrix_r13'),

        (2*(pl.col('rot_x')*pl.col('rot_y') + pl.col('rot_w')*pl.col('rot_z'))).alias('rot_matrix_r21'),
        (1 - 2*(pl.col('rot_x')**2 + pl.col('rot_z')**2)).alias('rot_matrix_r22'),
        (2*(pl.col('rot_y')*pl.col('rot_z') - pl.col('rot_w')*pl.col('rot_x'))).alias('rot_matrix_r23'),

        (2*(pl.col('rot_x')*pl.col('rot_z') - pl.col('rot_w')*pl.col('rot_y'))).alias('rot_matrix_r31'),
        (2*(pl.col('rot_y')*pl.col('rot_z') + pl.col('rot_w')*pl.col('rot_x'))).alias('rot_matrix_r32'),
        (1 - 2*(pl.col('rot_x')**2 + pl.col('rot_y')**2)).alias('rot_matrix_r33'),
    ])

    df = df.with_columns(rot_matrix_exprs)
    return df.to_pandas()

def angular_velocity_features(df):
    """Derive angular velocity from quaternion derivatives"""
    df = pl.from_pandas(df)

    angular_vel_exprs = []


    angular_vel_exprs.extend([
        pl.col('rot_w').diff().over('sequence_id').alias('rot_w_dot'),
        pl.col('rot_x').diff().over('sequence_id').alias('rot_x_dot'),
        pl.col('rot_y').diff().over('sequence_id').alias('rot_y_dot'),
        pl.col('rot_z').diff().over('sequence_id').alias('rot_z_dot'),
    ])

    df = df.with_columns(angular_vel_exprs)

    angular_vel_components = []
    angular_vel_components.extend([
        (2 * (-pl.col('rot_x')*pl.col('rot_w_dot') + pl.col('rot_w')*pl.col('rot_x_dot') +
              pl.col('rot_y')*pl.col('rot_z_dot') - pl.col('rot_z')*pl.col('rot_y_dot'))).alias('angular_vel_x'),
        (2 * (-pl.col('rot_y')*pl.col('rot_w_dot') + pl.col('rot_w')*pl.col('rot_y_dot') +
              pl.col('rot_z')*pl.col('rot_x_dot') - pl.col('rot_x')*pl.col('rot_z_dot'))).alias('angular_vel_y'),
        (2 * (-pl.col('rot_z')*pl.col('rot_w_dot') + pl.col('rot_w')*pl.col('rot_z_dot') +
              pl.col('rot_x')*pl.col('rot_y_dot') - pl.col('rot_y')*pl.col('rot_x_dot'))).alias('angular_vel_z'),
    ])

    df = df.with_columns(angular_vel_components)

    angular_vel_derived = []
    angular_vel_derived.extend([
        (pl.col('angular_vel_x')**2 + pl.col('angular_vel_y')**2 + pl.col('angular_vel_z')**2).sqrt().alias('angular_speed'),

        pl.col('angular_vel_x').diff().over('sequence_id').alias('angular_accel_x'),
        pl.col('angular_vel_y').diff().over('sequence_id').alias('angular_accel_y'),
        pl.col('angular_vel_z').diff().over('sequence_id').alias('angular_accel_z'),

        pl.max_horizontal([
            pl.col('angular_vel_x').abs(),
            pl.col('angular_vel_y').abs(),
            pl.col('angular_vel_z').abs()
        ]).alias('dominant_angular_vel'),
    ])

    df = df.with_columns(angular_vel_derived)

    angular_final = []
    angular_final.extend([
        (pl.col('angular_accel_x')**2 + pl.col('angular_accel_y')**2 + pl.col('angular_accel_z')**2).sqrt().alias('angular_accel_magnitude'),

        pl.col('angular_accel_x').diff().over('sequence_id').alias('angular_jerk_x'),
        pl.col('angular_accel_y').diff().over('sequence_id').alias('angular_jerk_y'),
        pl.col('angular_accel_z').diff().over('sequence_id').alias('angular_jerk_z'),
    ])

    df = df.with_columns(angular_final)
    return df.to_pandas()


def temporal_features(df):
    df = pl.from_pandas(df)

    temporal_exprs = []

        # Normalize sequence counter to 0-1 range per sequence
    df = df.with_columns([
        ((pl.col('sequence_counter') - pl.col('sequence_counter').min().over('sequence_id')) /
         (pl.col('sequence_counter').max().over('sequence_id') - pl.col('sequence_counter').min().over('sequence_id') + CONFIG.ERR))
        .alias('normalized_position')
    ])


    for col in imu_cols:
        temporal_exprs.extend([
            pl.col(col).diff().over('sequence_id').alias(f'{col}_velocity'),

            (pl.col(col) / (pl.col(col).shift(1).over('sequence_id').abs() + CONFIG.ERR)).alias(f'{col}_pct_vs_prev'),
            (pl.col(col) / (pl.col(col).shift(-1).over('sequence_id').abs() + CONFIG.ERR)).alias(f'{col}_pct_vs_next'),
        ])

    df = df.with_columns(temporal_exprs)

    acc_exprs = []
    for col in imu_cols:
        acc_exprs.extend([
            pl.col(col).diff(n=4).over('sequence_id').alias(f'{col}_snap'),
            pl.col(col).diff(n=5).over('sequence_id').alias(f'{col}_crackle'),
            pl.col(col).diff(n=6).over('sequence_id').alias(f'{col}_pop'),
        ])

    df = df.with_columns(acc_exprs)

    peak_exprs = []
    for col in imu_cols:
        peak_exprs.extend([
            ((pl.col(f'{col}_pct_vs_prev').abs() > 1.2) &
             (pl.col(f'{col}_pct_vs_next').abs() > 1.2)).alias(f'{col}_is_peak'),
        ])

    df = df.with_columns(peak_exprs)

    agg_temporal_exprs = []
    for col in imu_cols:
        agg_temporal_exprs.extend([
            pl.col(f'{col}_velocity').abs().mean().over('sequence_id').alias(f'{col}_avg_velocity'),
            pl.col(f'{col}_is_peak').sum().over('sequence_id').alias(f'{col}_peak_count'),
        ])

    df = df.with_columns(agg_temporal_exprs)

    return df.to_pandas()


def position_features(df):
    """Features based on position within sequence"""
    df = pl.from_pandas(df)

    position_exprs = []
    for col in imu_cols:
        position_exprs.extend([
            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_early_mean'),
            pl.when(pl.col('normalized_position') >= 0.2 , pl.col('normalized_position') < 0.4)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid_mean'),
            pl.when(pl.col('normalized_position') >= 0.4 , pl.col('normalized_position') < 0.6)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid2_mean'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid3_mean'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_late_mean'),

            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_early_std'),
            pl.when(pl.col('normalized_position') >= 0.2 , pl.col('normalized_position') < 0.4)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid_std'),
            pl.when(pl.col('normalized_position') >= 0.4 , pl.col('normalized_position') < 0.6)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid2_std'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid3_std'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_late_std'),


            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(f'{col}_velocity')).abs().mean().over('sequence_id').alias(f'{col}_early_velocity_mean'),
            pl.when(pl.col('normalized_position') >= 0.2 , pl.col('normalized_position') < 0.4)
              .then(pl.col(f'{col}_velocity')).abs().mean().over('sequence_id').alias(f'{col}_mid_velocity_mean'),
            pl.when(pl.col('normalized_position') >= 0.4 , pl.col('normalized_position') < 0.6)
              .then(pl.col(f'{col}_velocity')).abs().mean().over('sequence_id').alias(f'{col}_mid2_velocity_mean'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(f'{col}_velocity')).abs().mean().over('sequence_id').alias(f'{col}_mid3_velocity_mean'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(f'{col}_velocity')).abs().mean().over('sequence_id').alias(f'{col}_late_velocity_mean'),

            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(f'{col}_velocity')).abs().std().over('sequence_id').alias(f'{col}_early_velocity_std'),
            pl.when(pl.col('normalized_position') >= 0.2 , pl.col('normalized_position') < 0.4)
              .then(pl.col(f'{col}_velocity')).abs().std().over('sequence_id').alias(f'{col}_mid_velocity_std'),
            pl.when(pl.col('normalized_position') >= 0.4 , pl.col('normalized_position') < 0.6)
              .then(pl.col(f'{col}_velocity')).abs().std().over('sequence_id').alias(f'{col}_mid2_velocity_std'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(f'{col}_velocity')).abs().std().over('sequence_id').alias(f'{col}_mid3_velocity_std'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(f'{col}_velocity')).abs().std().over('sequence_id').alias(f'{col}_late_velocity_std'),

            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col).pow(2)).sum().over('sequence_id').alias(f'{col}_early_energy'),
            pl.when(pl.col('normalized_position') >= 0.2 , pl.col('normalized_position') < 0.4)
              .then(pl.col(col).pow(2)).sum().over('sequence_id').alias(f'{col}_mid_energy'),
            pl.when(pl.col('normalized_position') >= 0.4 , pl.col('normalized_position') < 0.6)
              .then(pl.col(col).pow(2)).sum().over('sequence_id').alias(f'{col}_mid2_energy'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col).pow(2)).sum().over('sequence_id').alias(f'{col}_mid3_energy'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col).pow(2)).sum().over('sequence_id').alias(f'{col}_late_energy'),

            pl.when(pl.col('normalized_position') >= 0.9)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_very_late_mean'),
            pl.when(pl.col('normalized_position') >= 0.9)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_very_late_std'),

          ])

    df = df.with_columns(position_exprs)

    position_ratio_exprs = []
    for col in imu_cols:
        position_ratio_exprs.extend([
            (pl.col(f'{col}_early_mean') / pl.col(f'{col}_late_mean')+CONFIG.ERR).alias(f'{col}_early_late_mean_ratio'),
            (pl.col(f'{col}_early_std') / pl.col(f'{col}_late_std')+CONFIG.ERR).alias(f'{col}_early_late_std_ratio'),
            (pl.col(f'{col}_early_energy') / pl.col(f'{col}_late_energy')+CONFIG.ERR).alias(f'{col}_early_late_energy_ratio'),
        ])

    df = df.with_columns(position_ratio_exprs)

    return df.to_pandas()


def zero_crossing_features(df):
    """
    Creates and compares three different methods for identifying "significant change".

    1.  vs_value: Change relative to the current signal value.
    2.  vs_std: Change relative to the signal's standard deviation over the sequence.
    3.  vs_mean: Change relative to the signal's mean over the sequence.

    For each method, it generates a '{col}_flag_...' and a '{col}_rate_...' column.
    """
    df_pl = pl.from_pandas(df)


    prereq_exprs = []
    for col in imu_cols:
        prereq_exprs.extend([
            pl.col(col).diff().over('sequence_id').abs().alias(f'{col}_diff_abs'),
            pl.col(col).mean().over('sequence_id').alias(f'{col}_mean'),
            pl.col(col).std().over('sequence_id').alias(f'{col}_std'),

            pl.col(col).filter(pl.col('normalized_position') >= 0.6)
              .mean().over('sequence_id').alias(f'{col}_late_mean'),

            pl.col(col).filter(pl.col('normalized_position') >= 0.6)
              .std().over('sequence_id').alias(f'{col}_late_std'),

        ])

    df_pl = df_pl.with_columns(prereq_exprs)

    threshold_exprs = []
    for col in imu_cols:
        late_phase_diffs = pl.col(f'{col}_diff_abs').filter(pl.col('normalized_position') >= 0.6)

        threshold_exprs.extend([
            late_phase_diffs.mean().over('sequence_id').alias(f'{col}_mean_of_diffs_threshold'),
            late_phase_diffs.std().over('sequence_id').alias(f'{col}_std_of_diffs_threshold'),
        ])

    df_pl = df_pl.with_columns(threshold_exprs)

    feature_exprs = []
    for col in imu_cols:
        # # Method 1: Relative to Current Value
        # flag_vs_value = (pl.col(f'{col}_diff_abs') > pl.col(col).abs() * 0.1).cast(pl.Int32).alias(f'{col}_flag_vs_value')
        # zero_crossing_rate = pl.when(pl.col('normalized_position') > 0.6).then(flag_vs_value).mean().over('sequence_id').alias(f'{col}_gesture_zero_crossing_rate')

        # # Method 2: Relative to Standard Deviation
        # flag_vs_std = (pl.col(f'{col}_diff_abs') > pl.col(f'{col}_std')).cast(pl.Int32).alias(f'{col}_flag_vs_std')
        # zero_crossing_rate = pl.when(pl.col('normalized_position') > 0.6).then(flag_vs_std).mean().over('sequence_id').alias(f'{col}_gesture_zero_crossing_rate')

        # Method 3: Relative to Mean
        # flag_vs_mean = (pl.col(f'{col}_diff_abs') > pl.col(f'{col}_mean').abs() * 0.1).cast(pl.Int32).alias(f'{col}_flag_vs_mean')
        # zero_crossing_rate = pl.when(pl.col('normalized_position') > 0.6).then(flag_vs_mean).mean().over('sequence_id').alias(f'{col}_gesture_zero_crossing_rate')

        # flag_vs_local_late_mean = (pl.col(f'{col}_diff_abs') > pl.col(f'{col}_late_mean').abs() * 0.1).cast(pl.Int32)
        # late_zero_crossing_rate_mean = pl.when(pl.col('normalized_position') > 0.6).then(flag_vs_local_late_mean).mean().over('sequence_id').alias(f'{col}_late_gesture_zero_crossing_rate_mean')

        flag_vs_local_late_std = (pl.col(f'{col}_diff_abs') > pl.col(f'{col}_late_std')).cast(pl.Int32)
        late_zero_crossing_rate_std = pl.when(pl.col('normalized_position') >= 0.6).then(flag_vs_local_late_std).mean().over('sequence_id').alias(f'{col}_late_gesture_zero_crossing_rate_std')

        flag_is_dwelling = (pl.col(f'{col}_diff_abs') < pl.col(f'{col}_late_std')).cast(pl.Int32)
        dwell_time = pl.when(pl.col('normalized_position') >= 0.6).then(flag_is_dwelling).sum().over('sequence_id').alias(f'{col}_dwell_time')


        # flag_vs_mean_diff = (pl.col(f'{col}_diff_abs') > pl.col(f'{col}_mean_of_diffs_threshold')).cast(pl.Int32)
        # rate_vs_mean_diff = pl.when(pl.col('normalized_position') >= 0.6).then(flag_vs_mean_diff).mean().over('sequence_id').alias(f'{col}_rate_vs_mean_diff')

        # flag_vs_std_diff = (pl.col(f'{col}_diff_abs') > pl.col(f'{col}_std_of_diffs_threshold')).cast(pl.Int32)
        # rate_vs_std_diff = pl.when(pl.col('normalized_position') >= 0.6).then(flag_vs_std_diff).mean().over('sequence_id').alias(f'{col}_rate_vs_std_diff')

        feature_exprs.extend([
            late_zero_crossing_rate_std,
            # dwell_time,
        ])

    df_pl = df_pl.with_columns(feature_exprs)

    return df_pl.to_pandas()


def cross_axis_correlation_features(df):
    """Correlation and coordination between different axes"""
    df = pl.from_pandas(df)

    corr_exprs = []

    axis_pairs = [('acc_x', 'acc_y'), ('acc_x', 'acc_z'), ('acc_y', 'acc_z')]
    for axis1, axis2 in axis_pairs:
        corr_exprs.append(
            pl.corr(pl.col(axis1), pl.col(axis2)).over('sequence_id').alias(f'{axis1}_{axis2}_corr')
        )

    euler_pairs = [('euler_roll', 'euler_pitch'), ('euler_roll', 'euler_yaw'), ('euler_pitch', 'euler_yaw')]
    for axis1, axis2 in euler_pairs:
        corr_exprs.append(
            pl.corr(pl.col(axis1), pl.col(axis2)).over('sequence_id').alias(f'{axis1}_{axis2}_corr')
        )

    df = df.with_columns(corr_exprs)
    return df.to_pandas()


# print("CASTING OBJECT TYPES")
# train = cast_to_object(train)

# print("MAG FEATURES")
# train = mag_features(train)

# print("ROTATION MATRIX FEATURES")
# train = rotation_matrix_features(train)

# print("ANGULAR VELOCITY FEATURES")
# train = angular_velocity_features(train)

# print("AGGREGATION FEATURES")
# train = aggregation_features(train)

# print("TEMPORAL FEATURES")
# train = temporal_features(train)

# print("POSITION FEATURES")
# train = position_features(train)

# print("ZERO CROSSING RATES")
# train = zero_crossing_features(train)

# print("CORR FEATURES")
# train = cross_axis_correlation_features(train)

def apply_feature_engineering(df):
    print("  Applying feature engineering...")
    df = cast_to_object(df)
    df = mag_features(df)
    df = rotation_matrix_features(df)
    df = angular_velocity_features(df)
    df = aggregation_features(df)
    df = temporal_features(df)
    df = position_features(df)
    df = zero_crossing_features(df)
    df = cross_axis_correlation_features(df)
    return df

train = apply_feature_engineering(train)
# test = apply_feature_engineering(test)

for i in imu_cols:
  FEATURES.remove(i)


In [ ]:
print(train.shape)

In [ ]:
train.phase.unique()

In [ ]:
train.sequence_id.iloc[1]

In [ ]:
len(FEATURES), train.shape

## TOF THM FEATURES

In [ ]:
thm_cols = [
    "thm_1", "thm_2", "thm_3", "thm_4", "thm_5",

    "thm_12_diff", "thm_13_diff", "thm_14_diff",
    "thm_15_diff", "thm_23_diff", "thm_24_diff", "thm_25_diff",
    "thm_34_diff", "thm_35_diff", "thm_45_diff",

]

tof_cols = [f"tof_{i}_v{j}" for i in range(1, 6) for j in range(64)]

tof_diff_cols = [f"tof_{i}{j}_mean_diff" for i in range(1, 6) for j in range(i+1, 6) if i != j]
tof_diff_cols += [f"tof_{i}{j}_std_diff" for i in range(1, 6) for j in range(i+1, 6) if i != j]

In [ ]:
def thermopile_features(df):
    """Extract features from thermopile sensors (temperature)"""
    df = pl.from_pandas(df)

    thm_diff_exprs = []
    thm_diff_exprs.extend([
        (pl.col('thm_1') - pl.col('thm_2')).abs().alias('thm_12_diff'),
        (pl.col('thm_1') - pl.col('thm_3')).abs().alias('thm_13_diff'),
        (pl.col('thm_1') - pl.col('thm_4')).abs().alias('thm_14_diff'),
        (pl.col('thm_1') - pl.col('thm_5')).abs().alias('thm_15_diff'),
        (pl.col('thm_2') - pl.col('thm_3')).abs().alias('thm_23_diff'),
        (pl.col('thm_2') - pl.col('thm_4')).abs().alias('thm_24_diff'),
        (pl.col('thm_2') - pl.col('thm_5')).abs().alias('thm_25_diff'),
        (pl.col('thm_3') - pl.col('thm_4')).abs().alias('thm_34_diff'),
        (pl.col('thm_3') - pl.col('thm_5')).abs().alias('thm_35_diff'),
        (pl.col('thm_4') - pl.col('thm_5')).abs().alias('thm_45_diff'),

    ])

    df = df.with_columns(thm_diff_exprs)

    thm_exprs = []
    for col in thm_cols:
        thm_exprs.extend([
            pl.col(col).mean().over('sequence_id').alias(f'{col}_seq_mean'),
            pl.col(col).std().over('sequence_id').alias(f'{col}_seq_std'),
            pl.col(col).min().over('sequence_id').alias(f'{col}_seq_min'),
            pl.col(col).max().over('sequence_id').alias(f'{col}_seq_max'),

            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_early_temp_mean'),
            pl.when(pl.col('normalized_position') >= 0.2 , pl.col('normalized_position') < 0.4)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid_temp_mean'),
            pl.when(pl.col('normalized_position') >= 0.4 , pl.col('normalized_position') < 0.6)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid2_temp_mean'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid3_temp_mean'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_late_temp_mean'),

            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_early_temp_std'),
            pl.when(pl.col('normalized_position') >= 0.2 , pl.col('normalized_position') < 0.4)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid_temp_std'),
            pl.when(pl.col('normalized_position') >= 0.4 , pl.col('normalized_position') < 0.6)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid2_temp_std'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid3_temp_std'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_late_temp_std'),


            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_early_temp_max'),
            pl.when(pl.col('normalized_position') >= 0.2 , pl.col('normalized_position') < 0.4)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_mid_temp_max'),
            pl.when(pl.col('normalized_position') >= 0.4 , pl.col('normalized_position') < 0.6)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_mid2_temp_max'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_mid3_temp_max'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_late_temp_max'),


            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col).diff().max().over('sequence_id').alias(f'{col}_late_max_heating_rate')),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col).diff().min().over('sequence_id').alias(f'{col}_late_max_cooling_rate')),

            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col).diff()).mean().over('sequence_id').alias(f'{col}_late_temp_rate'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col).diff()).std().over('sequence_id').alias(f'{col}_late_temp_rate_std'),

            pl.col(col).diff().diff().over('sequence_id').alias(f'{col}_temp_acceleration'),

        ])

    df = df.with_columns(thm_exprs)

    return df.to_pandas()


def tof_features(df):
    """Extract features from time-of-flight sensors (proximity/distance)"""
    df = pl.from_pandas(df)

    tof_sensor_exprs = []

    for sensor_idx in range(1, 6):  # 5 ToF sensors
        pixel_cols = [f'tof_{sensor_idx}_v{i}' for i in range(64)]

        tof_sensor_exprs.extend([
            pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in pixel_cols])
              .list.mean().alias(f'tof_{sensor_idx}_mean_distance'),

            pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in pixel_cols])
              .list.std().alias(f'tof_{sensor_idx}_std_distance'),
        ])

    df = df.with_columns(tof_sensor_exprs)

    return df.to_pandas()

def tof_cross_sensor_features(df):
    """Cross-sensor ToF features similar to thermopile differences"""
    df = pl.from_pandas(df)

    tof_diff_exprs = []
    tof_diff_exprs.extend([
        (pl.col('tof_1_mean_distance') - pl.col('tof_2_mean_distance')).abs().alias('tof_12_mean_diff'),
        (pl.col('tof_1_mean_distance') - pl.col('tof_3_mean_distance')).abs().alias('tof_13_mean_diff'),
        (pl.col('tof_1_mean_distance') - pl.col('tof_4_mean_distance')).abs().alias('tof_14_mean_diff'),
        (pl.col('tof_1_mean_distance') - pl.col('tof_5_mean_distance')).abs().alias('tof_15_mean_diff'),
        (pl.col('tof_2_mean_distance') - pl.col('tof_3_mean_distance')).abs().alias('tof_23_mean_diff'),
        (pl.col('tof_2_mean_distance') - pl.col('tof_4_mean_distance')).abs().alias('tof_24_mean_diff'),
        (pl.col('tof_2_mean_distance') - pl.col('tof_5_mean_distance')).abs().alias('tof_25_mean_diff'),
        (pl.col('tof_3_mean_distance') - pl.col('tof_4_mean_distance')).abs().alias('tof_34_mean_diff'),
        (pl.col('tof_3_mean_distance') - pl.col('tof_5_mean_distance')).abs().alias('tof_35_mean_diff'),
        (pl.col('tof_4_mean_distance') - pl.col('tof_5_mean_distance')).abs().alias('tof_45_mean_diff'),
    ])

    df = df.with_columns(tof_diff_exprs)

    overall_tof_exprs = []
    overall_tof_exprs.extend([
        # Overall proximity signature
        ((pl.col('tof_1_mean_distance') + pl.col('tof_2_mean_distance') + pl.col('tof_3_mean_distance') +
          pl.col('tof_4_mean_distance') + pl.col('tof_5_mean_distance')) / 5).alias('tof_overall_mean_distance'),
    ])

    df = df.with_columns(overall_tof_exprs)
    return df.to_pandas()

def tof_sequence_features(df):
    """Sequence-level ToF features with phase analysis"""
    df = pl.from_pandas(df)

    tof_diff_cols = [col for col in df.columns if 'tof_' in col and '_diff' in col]
    # tof_overall_cols = ['tof_overall_mean_distance', 'tof_closest_sensor_distance',
    #                    'tof_furthest_sensor_distance', 'tof_proximity_spread', 'tof_spatial_variance']

    seq_exprs = []

    for col in ['tof_1_mean_distance', 'tof_2_mean_distance', 'tof_3_mean_distance',
                'tof_4_mean_distance', 'tof_5_mean_distance'] + tof_diff_cols:
        seq_exprs.extend([
            pl.col(col).mean().over('sequence_id').alias(f'{col}_seq_mean'),
            pl.col(col).std().over('sequence_id').alias(f'{col}_seq_std'),
            pl.col(col).min().over('sequence_id').alias(f'{col}_seq_min'),
            pl.col(col).max().over('sequence_id').alias(f'{col}_seq_max'),

            # Phase-specific features
            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_early_mean'),
            pl.when(pl.col('normalized_position') >= 0.2, pl.col('normalized_position') < 0.4)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid_mean'),
            pl.when(pl.col('normalized_position') >= 0.4, pl.col('normalized_position') < 0.6)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid2_mean'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_mid3_mean'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col)).mean().over('sequence_id').alias(f'{col}_late_mean'),

            # Late phase proximity dynamics (critical for BFRB)
            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_early_std'),
            pl.when(pl.col('normalized_position') >= 0.2, pl.col('normalized_position') < 0.4)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid_std'),
            pl.when(pl.col('normalized_position') >= 0.4, pl.col('normalized_position') < 0.6)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid2_std'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_mid3_std'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col)).std().over('sequence_id').alias(f'{col}_late_std'),

            pl.when(pl.col('normalized_position') < 0.2)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_early_max'),
            pl.when(pl.col('normalized_position') >= 0.2, pl.col('normalized_position') < 0.4)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_mid_max'),
            pl.when(pl.col('normalized_position') >= 0.4, pl.col('normalized_position') < 0.6)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_mid2_max'),
            pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_mid3_max'),
            pl.when(pl.col('normalized_position') >= 0.8)
              .then(pl.col(col)).max().over('sequence_id').alias(f'{col}_late_max'),


            # pl.when(pl.col('normalized_position') >= 0.8)
            #   .then(pl.col(col)).min().over('sequence_id').alias(f'{col}_late_min'),
        ])

    df = df.with_columns(seq_exprs)
    return df.to_pandas()

def tof_spatial_structure_features(df):
    """Extract spatial structure features from 8x8 ToF grids"""
    df = pl.from_pandas(df)

    spatial_exprs = []

    for sensor_idx in range(1, 6):  # 5 ToF sensors
        horizontal_edges = []
        vertical_edges = []


        for row in range(7):  # 7 transitions between 8 rows
            row_start = row * 8
            next_row_start = (row + 1) * 8

            for col in range(8):
                pixel1 = f"tof_{sensor_idx}_v{row_start + col}"
                pixel2 = f"tof_{sensor_idx}_v{next_row_start + col}"

                horizontal_edges.append(
                    pl.when((pl.col(pixel1) != -1) & (pl.col(pixel2) != -1))
                    .then((pl.col(pixel1) - pl.col(pixel2)).abs())
                )
        for row in range(8):
            for col in range(7):  
                pixel1 = f'tof_{sensor_idx}_v{row * 8 + col}'
                pixel2 = f'tof_{sensor_idx}_v{row * 8 + col + 1}'

                vertical_edges.append(
                    pl.when((pl.col(pixel1) != -1) & (pl.col(pixel2) != -1))
                    .then((pl.col(pixel1) - pl.col(pixel2)).abs())
                )

        spatial_exprs.extend([
            pl.concat_list(horizontal_edges).list.mean().mean().over('sequence_id')
            .alias(f'tof_{sensor_idx}_horizontal_edge_strength_seq'),

            pl.concat_list(vertical_edges).list.mean().mean().over('sequence_id')
            .alias(f'tof_{sensor_idx}_vertical_edge_strength_seq'),

            pl.when(pl.col('normalized_position') >= 0.8)
            .then(pl.concat_list(horizontal_edges).list.mean())
            .mean().over('sequence_id')
            .alias(f'tof_{sensor_idx}_horizontal_edge_strength_late'),

            pl.when(pl.col('normalized_position') >= 0.8)
            .then(pl.concat_list(vertical_edges).list.mean())
            .mean().over('sequence_id')
            .alias(f'tof_{sensor_idx}_vertical_edge_strength_late'),

            pl.when((pl.col(f'tof_{sensor_idx}_v27') != -1) &
                   (pl.col(f'tof_{sensor_idx}_v28') != -1) &
                   (pl.col(f'tof_{sensor_idx}_v35') != -1) &
                   (pl.col(f'tof_{sensor_idx}_v36') != -1))
            .then((pl.col(f'tof_{sensor_idx}_v27') + pl.col(f'tof_{sensor_idx}_v28') +
                  pl.col(f'tof_{sensor_idx}_v35') + pl.col(f'tof_{sensor_idx}_v36')) / 4)
            .mean().over('sequence_id')
            .alias(f'tof_{sensor_idx}_center_pixels_seq_mean'),

            # Corner pixel activity
            pl.when((pl.col(f'tof_{sensor_idx}_v0') != -1) &
                   (pl.col(f'tof_{sensor_idx}_v7') != -1) &
                   (pl.col(f'tof_{sensor_idx}_v56') != -1) &
                   (pl.col(f'tof_{sensor_idx}_v63') != -1))
            .then((pl.col(f'tof_{sensor_idx}_v0') + pl.col(f'tof_{sensor_idx}_v7') +
                  pl.col(f'tof_{sensor_idx}_v56') + pl.col(f'tof_{sensor_idx}_v63')) / 4)
            .mean().over('sequence_id')
            .alias(f'tof_{sensor_idx}_corner_pixels_seq_mean'),
        ])

    df = df.with_columns(spatial_exprs)


    advanced_tof_exprs = []

    for sensor_idx in range(1, 6):
        pixel_cols = [f'tof_{sensor_idx}_v{i}' for i in range(64)]


        center_pixels = [f'tof_{sensor_idx}_v{i}' for i in [27, 28, 35, 36]]  # 2x2 center
        edge_pixels = [f'tof_{sensor_idx}_v{i}' for i in [0, 1, 2, 3, 4, 5, 6, 7,  # top row
                                                          56, 57, 58, 59, 60, 61, 62, 63,  # bottom row
                                                          8, 16, 24, 32, 40, 48,  # left column
                                                          15, 23, 31, 39, 47, 55]]  # right column

        advanced_tof_exprs.extend([
            (pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col))
                            for col in center_pixels]).list.mean() /
             (pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col))
                             for col in edge_pixels]).list.mean() + 0.001))
            .alias(f'tof_{sensor_idx}_center_edge_ratio'),


            (pl.concat_list([pl.when(pl.col(f'tof_{sensor_idx}_v{row*8 + col}') != -1)
                            .then(pl.col(f'tof_{sensor_idx}_v{row*8 + col}'))
                            for row in range(8) for col in range(4)]).list.mean() -
             pl.concat_list([pl.when(pl.col(f'tof_{sensor_idx}_v{row*8 + col}') != -1)
                            .then(pl.col(f'tof_{sensor_idx}_v{row*8 + col}'))
                            for row in range(8) for col in range(4, 8)]).list.mean()).abs()
            .alias(f'tof_{sensor_idx}_left_right_asymmetry'),
        ])

    df = df.with_columns(advanced_tof_exprs)

    return df.to_pandas()

# def cat_tof_regional_features_func(df, tof_mode="stats", include_regions=True):
#     """
#     Extract features from time-of-flight sensors with regional analysis

#     Args:
#         df: DataFrame with ToF data
#         tof_mode: "stats" for basic stats, "regions" for regional analysis, "multi" for multi-resolution
#         include_regions: Whether to include regional analysis features
#     """
#     df = pl.from_pandas(df)

#     tof_sensor_exprs = []

#     # Basic statistics for each ToF sensor (5 sensors total)
#     for sensor_idx in range(1, 6):
#         pixel_cols = [f'tof_{sensor_idx}_v{i}' for i in range(64)]

#         # # Basic stats (replace -1 with null for proper statistics)
#         # tof_sensor_exprs.extend([
#         #     pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in pixel_cols])
#         #       .list.mean().alias(f'tof_{sensor_idx}_mean'),
#         #     pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in pixel_cols])
#         #       .list.std().alias(f'tof_{sensor_idx}_std'),
#         #     pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in pixel_cols])
#         #       .list.min().alias(f'tof_{sensor_idx}_min'),
#         #     pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in pixel_cols])
#         #       .list.max().alias(f'tof_{sensor_idx}_max'),
#         # ])

#         # Regional Analysis - divide 8x8 grid into regions
#         if include_regions and tof_mode in ["regions", "multi"]:
#             # Different region modes
#             region_modes = [4] if tof_mode == "regions" else [4]

#             for mode in region_modes:
#                 region_size = 64 // mode  # pixels per region

#                 for region_idx in range(mode):
#                     start_pixel = region_idx * region_size
#                     end_pixel = (region_idx + 1) * region_size

#                     # Get pixel columns for this region
#                     region_pixel_cols = pixel_cols[start_pixel:end_pixel]

#                     # Calculate regional statistics
#                     tof_sensor_exprs.extend([
#                         pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in region_pixel_cols])
#                           .list.mean().alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mean'),
#                         pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in region_pixel_cols])
#                           .list.std().alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_std'),
#                         pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in region_pixel_cols])
#                           .list.min().alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_min'),
#                         pl.concat_list([pl.when(pl.col(col) != -1).then(pl.col(col)) for col in region_pixel_cols])
#                           .list.max().alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_max'),

#                     ])

#     df = df.with_columns(tof_sensor_exprs)

#     return df.to_pandas()


def cat_tof_regional_features_func(df, tof_mode="stats", include_regions=True):
    """
    Extract features from time-of-flight sensors with regional analysis across phases

    Args:
        df: DataFrame with ToF data
        tof_mode: "stats" for basic stats, "regions" for regional analysis, "multi" for multi-resolution
        include_regions: Whether to include regional analysis features
    """
    df = pl.from_pandas(df)

    tof_sensor_exprs = []

    for sensor_idx in range(1, 6):
        pixel_cols = [f'tof_{sensor_idx}_v{i}' for i in range(64)]

        if include_regions and tof_mode in ["regions", "multi"]:
            region_modes = [4] if tof_mode == "regions" else [4]

            for mode in region_modes:
                region_size = 64 // mode  

                for region_idx in range(mode):
                    start_pixel = region_idx * region_size
                    end_pixel = (region_idx + 1) * region_size

                    region_pixel_cols = pixel_cols[start_pixel:end_pixel]

                    region_values_expr = pl.concat_list([
                        pl.when(pl.col(col) != -1).then(pl.col(col))
                        for col in region_pixel_cols
                    ])

                    # Phase-specific regional statistics (5 phases)
                    # Early phase (normalized_position < 0.2)
                    # tof_sensor_exprs.extend([
                    #     pl.when(pl.col('normalized_position') < 0.2)
                    #       .then(region_values_expr.list.mean())
                    #       .mean().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_early_mean'),
                    #     pl.when(pl.col('normalized_position') < 0.2)
                    #       .then(region_values_expr.list.std())
                    #       .mean().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_early_std'),
                    #     pl.when(pl.col('normalized_position') < 0.2)
                    #       .then(region_values_expr.list.max())
                    #       .max().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_early_max'),

                    # ])

                    # # Mid phase (0.2 <= normalized_position < 0.4)
                    # tof_sensor_exprs.extend([
                    #     pl.when(pl.col('normalized_position') >= 0.2, pl.col('normalized_position') < 0.4)
                    #       .then(region_values_expr.list.mean())
                    #       .mean().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid_mean'),
                    #     pl.when(pl.col('normalized_position') >= 0.2, pl.col('normalized_position') < 0.4)
                    #       .then(region_values_expr.list.std())
                    #       .mean().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid_std'),
                    #     pl.when(pl.col('normalized_position') >= 0.2, pl.col('normalized_position') < 0.4)
                    #       .then(region_values_expr.list.max())
                    #       .max().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid_max'),
                    # ])

                    # # Mid2 phase (0.4 <= normalized_position < 0.6)
                    # tof_sensor_exprs.extend([
                    #     pl.when(pl.col('normalized_position') >= 0.4, pl.col('normalized_position') < 0.6)
                    #       .then(region_values_expr.list.mean())
                    #       .mean().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid2_mean'),
                    #     pl.when(pl.col('normalized_position') >= 0.4, pl.col('normalized_position') < 0.6)
                    #       .then(region_values_expr.list.std())
                    #       .mean().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid2_std'),
                    #     pl.when(pl.col('normalized_position') >= 0.4, pl.col('normalized_position') < 0.6)
                    #       .then(region_values_expr.list.max())
                    #       .max().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid2_max'),
                    # ])

                    # # Mid3 phase (0.6 <= normalized_position < 0.8)
                    # tof_sensor_exprs.extend([
                    #     pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
                    #       .then(region_values_expr.list.mean())
                    #       .mean().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid3_mean'),
                    #     pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
                    #       .then(region_values_expr.list.std())
                    #       .mean().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid3_std'),
                    #     pl.when(pl.col('normalized_position') >= 0.6, pl.col('normalized_position') < 0.8)
                    #       .then(region_values_expr.list.max())
                    #       .max().over('sequence_id')
                    #       .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_mid3_max'),
                    # ])

                    tof_sensor_exprs.extend([
                        pl.when(pl.col('normalized_position') >= 0.6)
                          .then(region_values_expr.list.mean())
                          .mean().over('sequence_id')
                          .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_late_mean'),
                        pl.when(pl.col('normalized_position') >= 0.6)
                          .then(region_values_expr.list.std())
                          .mean().over('sequence_id')
                          .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_late_std'),
                        pl.when(pl.col('normalized_position') >= 0.6)
                          .then(region_values_expr.list.max())
                          .max().over('sequence_id')
                          .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_late_max'),
                        pl.when(pl.col('normalized_position') >= 0.6)
                          .then(region_values_expr.list.min())
                          .min().over('sequence_id')
                          .alias(f'tof_{mode}_{sensor_idx}_region_{region_idx}_late_in'),

                    ])

    df = df.with_columns(tof_sensor_exprs)
    return df.to_pandas()

def cat_advanced_tof_features(df):
    """Advanced time-of-flight feature engineering"""
    df = pl.from_pandas(df)

    tof_advanced_exprs = []

    for sensor_idx in range(1, 6):
        pixel_cols = [f'tof_{sensor_idx}_v{i}' for i in range(64)]

        tof_advanced_exprs.extend([
            (pl.max_horizontal([pl.col(col) for col in pixel_cols[:16]]) -
             pl.min_horizontal([pl.col(col) for col in pixel_cols[:16]])).alias(f'tof_{sensor_idx}_contrast_q1'),

            (pl.max_horizontal([pl.col(col) for col in pixel_cols[16:32]]) -
             pl.min_horizontal([pl.col(col) for col in pixel_cols[16:32]])).alias(f'tof_{sensor_idx}_contrast_q2'),

            (pl.max_horizontal([pl.col(col) for col in pixel_cols[32:48]]) -
             pl.min_horizontal([pl.col(col) for col in pixel_cols[32:48]])).alias(f'tof_{sensor_idx}_contrast_q3'),

            (pl.max_horizontal([pl.col(col) for col in pixel_cols[48:64]]) -
             pl.min_horizontal([pl.col(col) for col in pixel_cols[48:64]])).alias(f'tof_{sensor_idx}_contrast_q4'),

        ])

    df = df.with_columns(tof_advanced_exprs)

    return df.to_pandas()


train = thermopile_features(train)

train = tof_features(train)
train = tof_cross_sensor_features(train)
train = tof_sequence_features(train)
train = tof_spatial_structure_features(train)
train = cat_tof_regional_features_func(train, tof_mode="regions")
# train = cat_advanced_tof_features(train)



thm_feature_cols = [col for col in train.columns if 'thm_' in col and col not in thm_cols]
# tof_feature_cols = [col for col in train.columns if 'tof_' in col and col not in tof_cols and col not in tof_diff_cols]
tof_feature_cols = [col for col in train.columns if 'tof_' in col and col not in tof_cols and col not in tof_diff_cols]


FEATURES_FULL = FEATURES + thm_feature_cols + tof_feature_cols
print(len(FEATURES_FULL))

## REDUCE MEMORY

In [ ]:
# def reduce_mem_usage(df, verbose=True):
#     numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#     start_mem = df.memory_usage().sum() / 1024**2 # calculate current memory usage

#     for col in df.columns:
#         col_type = df[col].dtype
#         if col_type in numerics: # check if column is numeric
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type).startswith('int'): # if integer
#                 # Check if data can be safely cast to smaller int types
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64) # Should already be this or smaller if loaded as int
#             else: # if float
#                 # Check if data can be safely cast to float32 (float16 often loses too much precision)
#                 if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 # else: # If not, keep as float64
#                 #     df[col] = df[col].astype(np.float64) # Already this type

#     end_mem = df.memory_usage().sum() / 1024**2
#     if verbose:
#         print(f'Memory usage reduced from {start_mem:.2f} MB to {end_mem:.2f} MB ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
#     return df

# print("Reducing memory for train_df:")
# train = reduce_mem_usage(train)
# print("\nReducing memory for test_df:")
# test = reduce_mem_usage(test)

# print("\nTrain DataFrame info after memory reduction:")
# train.info(memory_usage='deep')
# print("\nTest DataFrame info after memory reduction:")
# test.info(memory_usage='deep')

## METRIC

In [ ]:
"""
Hierarchical macro F1 metric for the CMI 2025 Challenge.

This script defines a single entry point `score(solution, submission, row_id_column_name)`
that the Kaggle metrics orchestrator will call.
It performs validation on submission IDs and computes a combined binary & multiclass F1 score.
"""

import pandas as pd
from sklearn.metrics import f1_score


class ParticipantVisibleError(Exception):
    """Errors raised here will be shown directly to the competitor."""
    pass


class CompetitionMetric:
    """Hierarchical macro F1 for the CMI 2025 challenge."""
    def __init__(self):
        self.target_gestures = [
            'Above ear - pull hair',
            'Cheek - pinch skin',
            'Eyebrow - pull hair',
            'Eyelash - pull hair',
            'Forehead - pull hairline',
            'Forehead - scratch',
            'Neck - pinch skin',
            'Neck - scratch',
        ]
        self.non_target_gestures = [
            'Write name on leg',
            'Wave hello',
            'Glasses on/off',
            'Text on phone',
            'Write name in air',
            'Feel around in tray and pull out an object',
            'Scratch knee/leg skin',
            'Pull air toward your face',
            'Drink from bottle/cup',
            'Pinch knee/leg skin'
        ]
        self.all_classes = self.target_gestures + self.non_target_gestures

    def calculate_hierarchical_f1(
        self,
        sol: pd.DataFrame,
        sub: pd.DataFrame
    ) -> float:

        # Validate gestures
        invalid_types = {i for i in sub['gesture'].unique() if i not in self.all_classes}
        if invalid_types:
            raise ParticipantVisibleError(
                f"Invalid gesture values in submission: {invalid_types}"
            )

        # Compute binary F1 (Target vs Non-Target)
        y_true_bin = sol['gesture'].isin(self.target_gestures).values
        y_pred_bin = sub['gesture'].isin(self.target_gestures).values
        f1_binary = f1_score(
            y_true_bin,
            y_pred_bin,
            pos_label=True,
            zero_division=0,
            average='binary'
        )

        # Build multi-class labels for gestures
        y_true_mc = sol['gesture'].apply(lambda x: x if x in self.target_gestures else 'non_target')
        y_pred_mc = sub['gesture'].apply(lambda x: x if x in self.target_gestures else 'non_target')

        # Compute macro F1 over all gesture classes
        f1_macro = f1_score(
            y_true_mc,
            y_pred_mc,
            average='macro',
            zero_division=0
        )

        print(f'f1_binary score: {f1_binary}')
        print(f'f1_macro score: {f1_macro}')

        return 0.5 * f1_binary + 0.5 * f1_macro


def score(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    row_id_column_name: str
) -> float:
    """
    Compute hierarchical macro F1 for the CMI 2025 challenge.

    Expected input:
      - solution and submission as pandas.DataFrame
      - Column 'sequence_id': unique identifier for each sequence
      - 'gesture': one of the eight target gestures or "Non-Target"

    This metric averages:
    1. Binary F1 on SequenceType (Target vs Non-Target)
    2. Macro F1 on gesture (mapping non-targets to "Non-Target")

    Raises ParticipantVisibleError for invalid submissions,
    including invalid SequenceType or gesture values.


    Examples
    --------
    >>> import pandas as pd
    >>> row_id_column_name = "id"
    >>> solution = pd.DataFrame({'id': range(4), 'gesture': ['Eyebrow - pull hair']*4})
    >>> submission = pd.DataFrame({'id': range(4), 'gesture': ['Forehead - pull hairline']*4})
    >>> score(solution, submission, row_id_column_name=row_id_column_name)
    0.5
    >>> submission = pd.DataFrame({'id': range(4), 'gesture': ['Text on phone']*4})
    >>> score(solution, submission, row_id_column_name=row_id_column_name)
    0.0
    >>> score(solution, solution, row_id_column_name=row_id_column_name)
    1.0
    """
    # Validate required columns
    for col in (row_id_column_name, 'gesture'):
        if col not in solution.columns:
            raise ParticipantVisibleError(f"Solution file missing required column: '{col}'")
        if col not in submission.columns:
            raise ParticipantVisibleError(f"Submission file missing required column: '{col}'")

    metric = CompetitionMetric()
    return metric.calculate_hierarchical_f1(solution, submission)

## MIXUP

In [ ]:
# def feature_mixup_for_catboost(X_train, y_train, alpha=0.3, mix_prob=0.5):
#     """Apply mixup to engineered features for CatBoost"""
#     X_mixed = []
#     y_mixed = []
#     np.random.seed(42)

#     for i in range(len(X_train)):
#         if np.random.rand() < mix_prob:
#             # Apply mixup
#             j = np.random.randint(0, len(X_train))
#             lam = np.random.beta(alpha, alpha)

#             # Mix features (this works for statistical features)
#             x_mix = lam * X_train.iloc[i] + (1 - lam) * X_train.iloc[j]

#             # For labels, use the dominant class (hard decision)
#             if lam > 0.5:
#                 y_mix = y_train.iloc[i]
#             else:
#                 y_mix = y_train.iloc[j]

#             X_mixed.append(x_mix)
#             y_mixed.append(y_mix)
#         else:
#             # Keep original
#             X_mixed.append(X_train.iloc[i])
#             y_mixed.append(y_train.iloc[i])

    # return pd.DataFrame(X_mixed), pd.Series(y_mixed)

def feature_mixup_for_catboost(X_train, y_train, alpha=0.3, mix_prob=1.0):
    """
    Applies feature space mixup by creating new synthetic rows and adding them
    to the original dataset.

    Args:
        X_train (pd.DataFrame): Original feature data.
        y_train (pd.Series): Original label data.
        alpha (float): Beta distribution parameter for Mixup.
        augmentation_factor (float): The fraction of new data to generate.
                                     0.5 means increase dataset size by 50%.
    """
    np.random.seed(42)

    num_original_samples = len(X_train)
    num_new_samples = int(num_original_samples * mix_prob)

    if num_new_samples == 0:
        return X_train, y_train

    print(f"Original samples: {num_original_samples}. Adding {num_new_samples} mixed samples.")

    indices1 = np.random.randint(0, num_original_samples, num_new_samples)
    indices2 = np.random.randint(0, num_original_samples, num_new_samples)

    lams = np.random.beta(alpha, alpha, size=num_new_samples)


    X1 = X_train.iloc[indices1].values
    X2 = X_train.iloc[indices2].values
    X_new = lams[:, np.newaxis] * X1 + (1 - lams)[:, np.newaxis] * X2

    y1 = y_train.iloc[indices1].values
    y2 = y_train.iloc[indices2].values
    y_new = np.where(lams > 0.5, y1, y2)


    X_augmented_df = pd.DataFrame(X_new, columns=X_train.columns)
    y_augmented_series = pd.Series(y_new)


    X_final = pd.concat([X_train, X_augmented_df], ignore_index=True)
    y_final = pd.concat([y_train, y_augmented_series], ignore_index=True)

    return X_final, y_final


# def feature_mixup_for_catboost(X_train, y_train, sequence_ids, alpha=0.3, mix_prob=0.5):
#     """
#     Fast vectorized version of sequence-level mixup.

#     Args:
#         X_train (pd.DataFrame): Original feature data.
#         y_train (pd.Series): Original label data.
#         sequence_ids (pd.Series): Sequence ID for each row.
#         alpha (float): Beta distribution parameter for Mixup.
#         mix_prob (float): The fraction of new sequences to generate.
#     """
#     np.random.seed(42)

#     # Get unique sequences and their representative indices
#     unique_sequences = sequence_ids.unique()
#     num_original_sequences = len(unique_sequences)
#     num_new_sequences = int(num_original_sequences * mix_prob)

#     if num_new_sequences == 0:
#         return X_train, y_train

#     print(f"Original sequences: {num_original_sequences}. Adding {num_new_sequences} mixed sequences.")

#     # Create mapping from sequence to first occurrence index (vectorized)
#     first_occurrence_mask = ~sequence_ids.duplicated()
#     representative_data = X_train[first_occurrence_mask].copy()
#     representative_labels = y_train[first_occurrence_mask].copy()
#     representative_sequences = sequence_ids[first_occurrence_mask].copy()

#     # Create sequence to row count mapping (vectorized)
#     sequence_counts = sequence_ids.value_counts()

#     # Generate random pairs and mixing weights (all vectorized)
#     seq_indices1 = np.random.randint(0, num_original_sequences, num_new_sequences)
#     seq_indices2 = np.random.randint(0, num_original_sequences, num_new_sequences)
#     lams = np.random.beta(alpha, alpha, size=num_new_sequences)

#     # Get the actual sequence IDs for selected indices
#     selected_seqs1 = representative_sequences.iloc[seq_indices1].values
#     selected_seqs2 = representative_sequences.iloc[seq_indices2].values

#     # Mix features (fully vectorized)
#     X1 = representative_data.iloc[seq_indices1].values
#     X2 = representative_data.iloc[seq_indices2].values
#     X_mixed_base = lams[:, np.newaxis] * X1 + (1 - lams)[:, np.newaxis] * X2

#     # Mix labels (vectorized hard assignment)
#     y1 = representative_labels.iloc[seq_indices1].values
#     y2 = representative_labels.iloc[seq_indices2].values
#     y_mixed_base = np.where(lams > 0.5, y1, y2)

#     # Calculate row counts for mixed sequences (vectorized)
#     counts1 = sequence_counts[selected_seqs1].values
#     counts2 = sequence_counts[selected_seqs2].values
#     mixed_counts = ((counts1 + counts2) / 2).astype(int)

#     # Calculate total rows needed
#     total_new_rows = mixed_counts.sum()

#     # Pre-allocate arrays for maximum speed
#     X_new = np.empty((total_new_rows, X_train.shape[1]), dtype=X_train.dtypes.iloc[0])
#     y_new = np.empty(total_new_rows, dtype=y_train.dtype)

#     # Fill arrays using vectorized operations with repeat
#     current_idx = 0
#     for i, count in enumerate(mixed_counts):
#         end_idx = current_idx + count
#         # Use numpy repeat for each mixed sample
#         X_new[current_idx:end_idx] = np.repeat(X_mixed_base[i:i+1], count, axis=0)
#         y_new[current_idx:end_idx] = y_mixed_base[i]
#         current_idx = end_idx

#     # Convert to pandas (single operation)
#     X_new_df = pd.DataFrame(X_new, columns=X_train.columns)
#     y_new_series = pd.Series(y_new)

#     # Concatenate (single operation)
#     X_final = pd.concat([X_train, X_new_df], ignore_index=True)
#     y_final = pd.concat([y_train, y_new_series], ignore_index=True)

#     return X_final, y_final

def generate_mixed_samples(df, features, target, n_mixed=1000):
    """Generate mixed samples for CatBoost training"""
    mixed_samples = []

    for _ in range(n_mixed):
        idx1, idx2 = np.random.choice(len(df), 2, replace=False)
        lam = np.random.beta(0.3, 0.3)


        mixed_features = lam * df.iloc[idx1][features] + (1 - lam) * df.iloc[idx2][features]
        mixed_label = df.iloc[idx1][target] if lam > 0.5 else df.iloc[idx2][target]

        mixed_sample = mixed_features.copy()
        mixed_sample[target] = mixed_label
        mixed_sample['sequence_id'] = f"mixed_{len(mixed_samples)}"

        mixed_samples.append(mixed_sample)

    return pd.DataFrame(mixed_samples)


# def mix_time_series_sequences(seq1, seq2, lam):
#     """Mix two time series sequences"""
#     # Ensure same length
#     min_len = min(len(seq1), len(seq2))
#     seq1_cut = seq1[:min_len]
#     seq2_cut = seq2[:min_len]

#     # Linear interpolation
#     mixed_seq = lam * seq1_cut + (1 - lam) * seq2_cut

#     return mixed_seq

# def create_mixed_sequences(df, n_mixed=500):
#     """Create mixed sequences, then extract features"""
#     mixed_data = []

#     sequence_ids = df['sequence_id'].unique()

#     for _ in range(n_mixed):
#         # Sample two sequences
#         seq_id1, seq_id2 = np.random.choice(sequence_ids, 2, replace=False)
#         seq1_data = df[df['sequence_id'] == seq_id1]
#         seq2_data = df[df['sequence_id'] == seq_id2]

#         lam = np.random.beta(0.3, 0.3)

#         # Mix sensor readings
#         mixed_features = {}
#         for col in ['acc_x', 'acc_y', 'acc_z', 'euler_roll', 'euler_pitch', 'euler_yaw']:
#             if col in df.columns:
#                 mixed_col = lam * seq1_data[col].values + (1 - lam) * seq2_data[col].values
#                 mixed_features[col] = mixed_col

#         # Extract features from mixed sequence
#         mixed_seq_features = extract_sequence_features(mixed_features)

#         # Choose dominant label
#         label = seq1_data['gesture'].iloc[0] if lam > 0.5 else seq2_data['gesture'].iloc[0]
#         mixed_seq_features['gesture'] = label

#         mixed_data.append(mixed_seq_features)

#     return pd.DataFrame(mixed_data)

## CATBOOST

In [ ]:
import catboost
from catboost import CatBoostClassifier
print(catboost.__version__)

In [ ]:
from sklearn.preprocessing import LabelEncoder
import joblib

cat_params = {
    'objective': 'MultiClass',
    'eval_metric': 'MultiClass',
    'learning_rate': 0.03,
    'depth': 7,
    'random_seed': 42,
    'iterations': 1000,
    'task_type': 'GPU',
    # 'reg_lambda': 5,
    'verbose': 50,
}

le = LabelEncoder()
train['gesture'] = le.fit_transform(train['gesture'])
joblib.dump(le, 'label_encoder.pkl')

metric_calculator = CompetitionMetric()


In [ ]:
# train = train.groupby('sequence_id').last().reset_index()

In [ ]:
X_train = train[FEATURES_FULL]
y_train = train[CONFIG.TARGET]
X_train.head()

In [ ]:
%%time

import os
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import uuid
run_id  = uuid.uuid4()

os.makedirs('models_full', exist_ok=True)
n_splits=5

t_d = train_demographics
skf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (tr_idx, val_idx) in enumerate(
        skf.split(t_d, t_d[['adult_child', 'handedness', 'sex']])
    ):
    t_d.loc[val_idx, 'fold'] = fold

t_d['fold'] = t_d['fold'].astype(int)
print("Demographics fold distribution:\n", t_d['fold'].value_counts(), "\n")

train = train.merge(t_d[['subject', 'fold']], on='subject', how='left')

oof_preds_cat              = np.zeros(len(train), dtype=int)
oof_preds_proba_cat        = np.zeros((len(train), CONFIG.NUM_CLASSES))
oof_scores                 = []
all_fold_importances       = []

metric_calculator = CompetitionMetric()


for fold in range(n_splits):
    print(f"{'#'*10} Fold {fold+1} {'#'*10}")

    train_idx = train.index[train['fold'] != fold].tolist()
    valid_idx = train.index[train['fold'] == fold].tolist()

    X_train_orig = train.loc[train_idx, FEATURES_FULL].copy()
    y_train_orig = train.loc[train_idx, CONFIG.TARGET].copy()
    sequence_ids = train.loc[train_idx, 'sequence_id'].copy()

    # X_train = train.loc[train_idx, FEATURES].copy()
    # y_train = train.loc[train_idx, CONFIG.TARGET].copy()

    print(f"Shape before Mixup: {X_train_orig.shape}")
    print(f"Valid shape before Mixup: {len(valid_idx)}")

    X_train, y_train = feature_mixup_for_catboost(X_train_orig,
                                                  y_train_orig,
                                                  alpha=0.3, mix_prob=1.0)

    print(f"Shape after Mixup: {X_train.shape}")

    X_valid = train.loc[valid_idx, FEATURES_FULL].copy() #
    y_valid = train.loc[valid_idx, CONFIG.TARGET].copy() #

    print(f"  X_train shape: {X_train.shape}, X_valid shape: {X_valid.shape}")

    model = CatBoostClassifier(**cat_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        use_best_model=True,
        # cat_features=cat_features,
        verbose=50
    )
    model.save_model(f"/content/drive/MyDrive/cmi2025/models/cat_fold_{fold+1}_{run_id}.cbm")
    model.save_model(f"models_full/cat_fold_{fold+1}.cbm")


    all_fold_importances.append(model.get_feature_importance())

    fold_proba = model.predict_proba(X_valid)
    oof_preds_proba_cat[valid_idx] = fold_proba
    fold_preds = np.argmax(fold_proba, axis=1)

    y_valid_orig = le.inverse_transform(y_valid)
    preds_orig   = le.inverse_transform(fold_preds)

    temp_sol_df = pd.DataFrame({"gesture": y_valid_orig})
    temp_sub_df = pd.DataFrame({"gesture": preds_orig})
    fold_score  = metric_calculator.calculate_hierarchical_f1(temp_sol_df, temp_sub_df)

    oof_scores.append(fold_score)
    print(f"  Fold {fold+1} Score: {fold_score}\n")

print(f"Mean OOF Score: {np.mean(oof_scores):.4f}")
print(f"Std  OOF Score: {np.std(oof_scores):.4f}")

In [ ]:

print(f"Mean OOF Score: {np.mean(oof_scores):.4f}")
print(f"Std  OOF Score: {np.std(oof_scores):.4f}\n")

original_labels = le.inverse_transform(train[CONFIG.TARGET])
oof_preds_encoded = np.argmax(oof_preds_proba_cat, axis=1)
oof_preds_original = le.inverse_transform(oof_preds_encoded)

sol_df = pd.DataFrame({"gesture": original_labels})
sub_df = pd.DataFrame({"gesture": oof_preds_original})

np.save('oof_preds_cat.npy', oof_preds_original)
np.save('oof_preds_proba_cat.npy', oof_preds_proba_cat)

overall_oof = metric_calculator.calculate_hierarchical_f1(sol_df, sub_df)
print(f"Overall OOF Score: {overall_oof:.4f}")



## full features:

# Mean OOF Score: 0.8251
# Std  OOF Score: 0.0057

# f1_binary score: 0.9799055112614349
# f1_macro score: 0.670842812666158
# Overall OOF Score: 0.8254


## full features + tof 4 features:
# Mean OOF Score: 0.8235
# Std  OOF Score: 0.0058

# f1_binary score: 0.9796848455254779
# f1_macro score: 0.668053497970177
# Overall OOF Score: 0.8239


## full features + tof 4 features normalize edilmiş pozisyona göre:

# Mean OOF Score: 0.8258
# Std  OOF Score: 0.0078

# f1_binary score: 0.9798178957684467
# f1_macro score: 0.672360790621561
# Overall OOF Score: 0.8261

## full features + tof 4 features normalize edilmiş pozisyona göre sadece 0.6dan sonrası:

# Mean OOF Score: 0.8269
# Std  OOF Score: 0.0070

# f1_binary score: 0.979738011560022
# f1_macro score: 0.6742888403172628
# Overall OOF Score: 0.8270


## full features + tof 4 features normalize edilmiş pozisyona göre sadece 0.6dan sonrası:
# Mean OOF Score: 0.8286
# Std  OOF Score: 0.0125

# f1_binary score: 0.9817535833918712
# f1_macro score: 0.6763128311048723
# Overall OOF Score: 0.8290

## full features + tof 4 features normalize edilmiş pozisyona göre sadece 0.6dan sonrası ve active pixels feature:


## FEATURE IMPORTANCES ETC.

In [ ]:
import matplotlib.pyplot as plt

importances_matrix = np.vstack(all_fold_importances)  # shape = (n_folds, n_features)
mean_importances = np.mean(importances_matrix, axis=0)
std_importances  = np.std(importances_matrix, axis=0)

feat_imp_df = pd.DataFrame({
    "feature": FEATURES,
    "mean_importance": mean_importances,
    "std_importance": std_importances
})
feat_imp_df = feat_imp_df.sort_values(by="mean_importance", ascending=False).reset_index(drop=True)


features = feat_imp_df["feature"].tolist()
means    = feat_imp_df["mean_importance"].values
stds     = feat_imp_df["std_importance"].values

y_pos = list(range(len(features)))[::-1]

plt.figure(figsize=(10, max(6, len(features) * 0.3)))
plt.barh(
    y=y_pos,
    width=means[::-1],
    xerr=stds[::-1],
    align="center",
    ecolor="black",
    capsize=4
)
plt.yticks(y_pos, [features[i] for i in reversed(range(len(features)))])
plt.xlabel("Mean feature importance")
plt.title("Feature importances (mean ± std across folds)")
plt.gca().invert_yaxis()  # highest mean at the top
plt.tight_layout()
plt.show()


In [ ]:
feat_imp_df

In [ ]:
import uuid
import json
import os

log_path = "/content/drive/MyDrive/cmi2025/models/experiments_log.csv"

fold_columns = {f"fold_{i+1}_oof": oof_scores[i] for i in range(len(oof_scores))}

summary_columns = {
    "mean_oof_score":     np.mean(oof_scores),
    "std_oof_score":      np.std(oof_scores),
    "overall_oof_score":  overall_oof,
    "cv_method": skf,
    "no_features": len(FEATURES),
    # "cat_params": cat_params
}

features_column = {"features_used": json.dumps(FEATURES)}


cat_params_column = {"cat_params": cat_params}

row_dict = {
    "run_id": run_id,
    **fold_columns,
    **summary_columns,
    **features_column,
    **cat_params_column
}

if os.path.isfile(log_path):
    log_df = pd.read_csv(log_path)
    log_df = pd.concat([log_df, pd.DataFrame([row_dict])], ignore_index=True)
else:
    log_df = pd.DataFrame([row_dict])

log_df.to_csv(log_path, index=False)
print(f"Appended run {run_id} to {log_path}")

np.save('/content/drive/MyDrive/cmi2025/models/oof_preds_original_labels.npy', oof_preds_original)
np.save('/content/drive/MyDrive/cmi2025/models/oof_preds_cat.npy', oof_preds_cat)
target_gestures = [
    'Above ear - pull hair',
    'Cheek - pinch skin',
    'Eyebrow - pull hair',
    'Eyelash - pull hair',
    'Forehead - pull hairline',
    'Forehead - scratch',
    'Neck - pinch skin',
    'Neck - scratch',
]

non_target_gestures = [
    'Write name on leg',
    'Wave hello',
    'Glasses on/off',
    'Text on phone',
    'Write name in air',
    'Feel around in tray and pull out an object',
    'Scratch knee/leg skin',
    'Pull air toward your face',
    'Drink from bottle/cup',
    'Pinch knee/leg skin'
]

results_df = pd.DataFrame({
    'run_id': run_id,
    'target_encoded': train[CONFIG.TARGET],
    'target_original': original_labels,
    'pred_encoded': oof_preds_cat,
    'pred_original': oof_preds_original,
    'subject': train[CONFIG.SUBJECT],
    'sequence_id': train['sequence_id'],
    'correct_prediction': (original_labels == oof_preds_original)
})

results_df['target_category'] = results_df['target_original'].apply(
    lambda x: 'target' if x in target_gestures else 'non-target'
)
results_df['pred_category'] = results_df['pred_original'].apply(
    lambda x: 'target' if x in target_gestures else 'non-target'
)

results_df.to_csv('/content/drive/MyDrive/cmi2025/models/results.csv', index=False)
print(results_df.head())
sequence_results_df = results_df.groupby('sequence_id').last().reset_index()
sequence_results_df.to_csv('/content/drive/MyDrive/cmi2025/models/sequence_results.csv', index=False)
print(sequence_results_df.head())

In [ ]:
import matplotlib.pyplot as plt

def load_models():
    models = []
    for i in range(CONFIG.FOLDS):
        model = CatBoostClassifier()
        model.load_model(f'/content/models_full/cat_fold_{i+1}.pkl')
        models.append(model)
    return models

imu_models = load_models()

imu_models[0].plot_tree(tree_idx=0)

## FEATURE REMOVAL

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
import pickle
import os
from datetime import datetime

metric_calculator = CompetitionMetric()


# Feature groups to test (removing one at a time)
feature_groups_to_test = [
        '_avg_velocity',

        '_early_mean',
        '_mid_mean',
        '_mid2_mean',
        '_mid3_mean',
        '_late_mean',

        '_early_std',
        '_mid_std',
        '_mid2_std',
        '_mid3_std',
        '_late_std',

        '_early_velocity_mean',
        '_mid_velocity_mean',
        '_mid2_velocity_mean',
        '_mid3_velocity_mean',
        '_late_velocity_mean',

        '_early_velocity_std',
        '_mid_velocity_std',
        '_mid2_velocity_std',
        '_mid3_velocity_std',
        '_late_velocity_std',

        # f'{col}_early_energy',
        # f'{col}_mid_energy',
        # f'{col}_mid2_energy',
        # f'{col}_mid3_energy',
        '_late_energy',

        # f'{col}_early_gesture_oscillations', #
        # f'{col}_mid_gesture_oscillations', #
        # f'{col}_mid2_gesture_oscillations', #
        # f'{col}_mid3_gesture_oscillations', #
        # f'{col}_late_gesture_oscillations', #

        '_early_late_mean_ratio',
        '_early_late_std_ratio',
        '_early_late_energy_ratio',
]

def get_features_excluding_group(feature_suffix, original_features, imu_cols):
    """
    Get feature list excluding all features that end with the given suffix
    """
    excluded_features = []
    for col in imu_cols:
        excluded_features.append(f'{col}{feature_suffix}')

    # Filter out the excluded features
    filtered_features = [f for f in original_features if f not in excluded_features]
    return filtered_features

def train_and_evaluate_fold(feature_list, train_data, config, cat_params, le, metric_calculator):
    """
    Train model with given feature list and return OOF score
    """
    sgkf = StratifiedGroupKFold(n_splits=config.FOLDS, shuffle=True, random_state=42)
    groups = train_data[config.SUBJECT]

    oof_preds_cat = np.zeros(len(train_data), dtype=int)
    oof_scores = []

    for i, (train_idx, valid_idx) in enumerate(sgkf.split(train_data, train_data[config.TARGET], groups)):

        # TRAIN
        X_train = train_data.loc[train_idx, feature_list].copy()
        y_train = train_data.loc[train_idx, config.TARGET].copy()

        # VALID
        X_valid = train_data.loc[valid_idx, feature_list].copy()
        y_valid = train_data.loc[valid_idx, config.TARGET].copy()

        y_valid_original_labels = le.inverse_transform(y_valid)

        model = CatBoostClassifier(**cat_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            use_best_model=True,
            verbose=0  # Silent training for ablation study
        )

        fold_preds_proba = model.predict_proba(X_valid)
        fold_valid_preds_labels_encoded = np.argmax(fold_preds_proba, axis=1)
        oof_preds_cat[valid_idx] = fold_valid_preds_labels_encoded

        fold_valid_preds_original_labels = le.inverse_transform(fold_valid_preds_labels_encoded)

        temp_sol_df = pd.DataFrame({"gesture": y_valid_original_labels})
        temp_sub_df = pd.DataFrame({"gesture": fold_valid_preds_original_labels})

        fold_score = metric_calculator.calculate_hierarchical_f1(temp_sol_df, temp_sub_df)
        oof_scores.append(fold_score)

    # Calculate overall OOF score
    original_labels = le.inverse_transform(train_data[CONFIG.TARGET])
    oof_preds_original_labels = le.inverse_transform(oof_preds_cat)

    sol_df = pd.DataFrame({"gesture": original_labels})
    sub_df = pd.DataFrame({"gesture": oof_preds_original_labels})

    overall_oof = metric_calculator.calculate_hierarchical_f1(sol_df, sub_df)

    return overall_oof, np.mean(oof_scores), np.std(oof_scores)

# # Run feature ablation study
# print("Starting Feature Ablation Study...")
# print("="*50)

# # Get baseline score (with all features)
# print("Calculating baseline score with all features...")
# baseline_oof, baseline_mean, baseline_std = train_and_evaluate_fold(
#     FEATURES, train, CONFIG, cat_params, le, metric_calculator
# )

# print(f"Baseline Overall OOF Score: {baseline_oof:.6f}")
# print(f"Baseline Mean Fold Score: {baseline_mean:.6f} ± {baseline_std:.6f}")
# print("="*50)

# # Results storage
results = []
# results.append({
#     'feature_group': 'BASELINE_ALL_FEATURES',
#     'num_features': len(FEATURES),
#     'overall_oof': baseline_oof,
#     'mean_fold_score': baseline_mean,
#     'std_fold_score': baseline_std,
#     'score_drop': 0.0,
#     'removed_features': 0
# })

baseline_oof = 0.7491

# Test removing each feature group
for feature_suffix in feature_groups_to_test:
    print(f"Testing removal of features ending with: {feature_suffix}")

    # Get features excluding this group
    filtered_features = get_features_excluding_group(feature_suffix, FEATURES, imu_cols)

    # Calculate how many features were removed
    removed_count = len(FEATURES) - len(filtered_features)

    print(f"Removed {removed_count} features, using {len(filtered_features)} features")

    # Train and evaluate
    overall_oof, mean_score, std_score = train_and_evaluate_fold(
        filtered_features, train, CONFIG, cat_params, le, metric_calculator
    )

    score_drop = baseline_oof - overall_oof

    print(f"Overall OOF Score: {overall_oof:.6f}")
    print(f"Score Drop: {score_drop:.6f}")
    print("-" * 30)

    # Store results
    results.append({
        'feature_group': feature_suffix,
        'num_features': len(filtered_features),
        'overall_oof': overall_oof,
        'mean_fold_score': mean_score,
        'std_fold_score': std_score,
        'score_drop': score_drop,
        'removed_features': removed_count
    })

# Save results to file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"feature_ablation_results_{timestamp}.txt"

with open(filename, 'w') as f:
    f.write("Feature Ablation Study Results\n")
    f.write("="*50 + "\n")
    f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Total Features in Baseline: {len(FEATURES)}\n")
    f.write(f"Baseline Overall OOF Score: {baseline_oof:.6f}\n\n")

    f.write("Results Summary:\n")
    f.write("-" * 80 + "\n")
    f.write(f"{'Feature Group':<25} {'Num Features':<12} {'OOF Score':<12} {'Score Drop':<12} {'Removed':<8}\n")
    f.write("-" * 80 + "\n")

    for result in results:
        f.write(f"{result['feature_group']:<25} {result['num_features']:<12} "
                f"{result['overall_oof']:<12.6f} {result['score_drop']:<12.6f} {result['removed_features']:<8}\n")

    f.write("\n" + "="*50 + "\n")
    f.write("Detailed Results:\n\n")

    for result in results:
        f.write(f"Feature Group: {result['feature_group']}\n")
        f.write(f"  Number of Features Used: {result['num_features']}\n")
        f.write(f"  Features Removed: {result['removed_features']}\n")
        f.write(f"  Overall OOF Score: {result['overall_oof']:.6f}\n")
        f.write(f"  Mean Fold Score: {result['mean_fold_score']:.6f} ± {result['std_fold_score']:.6f}\n")
        f.write(f"  Score Drop from Baseline: {result['score_drop']:.6f}\n")
        if result['score_drop'] > 0:
            f.write(f"  Performance Impact: NEGATIVE (score decreased)\n")
        elif result['score_drop'] < 0:
            f.write(f"  Performance Impact: POSITIVE (score improved)\n")
        else:
            f.write(f"  Performance Impact: NEUTRAL\n")
        f.write("\n")

# Create summary DataFrame and display
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('score_drop', ascending=False)

print("\n" + "="*50)
print("FEATURE ABLATION STUDY COMPLETE")
print("="*50)
print(f"Results saved to: {filename}")
print("\nSummary (sorted by score drop):")
print(results_df[['feature_group', 'overall_oof', 'score_drop', 'removed_features']].to_string(index=False))

# Identify most impactful features
print(f"\nMost impactful feature groups (highest score drop when removed):")
top_impactful = results_df[results_df['feature_group'] != 'BASELINE_ALL_FEATURES'].head(3)
for _, row in top_impactful.iterrows():
    print(f"  {row['feature_group']}: -{row['score_drop']:.6f} score drop")

print(f"\nLeast impactful feature groups (lowest score drop when removed):")
least_impactful = results_df[results_df['feature_group'] != 'BASELINE_ALL_FEATURES'].tail(3)
for _, row in least_impactful.iterrows():
    print(f"  {row['feature_group']}: -{row['score_drop']:.6f} score drop")


## FEATURE ADDING

In [ ]:
from sklearn.preprocessing import LabelEncoder
import joblib

cat_params = {
    'objective': 'MultiClass',
    'eval_metric': 'MultiClass',
    'learning_rate': 0.03,
    'depth': 7,
    'random_seed': 42,
    'iterations': 1000,
    'task_type': 'GPU',
    # 'reg_lambda': 10,
    'verbose': 50,
}

le = LabelEncoder()
train['gesture'] = le.fit_transform(train['gesture'])
joblib.dump(le, 'label_encoder.pkl')

metric_calculator = CompetitionMetric()


feature_suffixes_to_add = [
        '_seq_mean',
        '_seq_std',
        '_seq_min',
        '_seq_max',
        '_seq_q10',
        '_seq_q90',
        '_seq_median',
        '_seq_iqr',
        '_seq_range',

        '_avg_velocity',
        '_max_velocity',
        '_avg_acceleration',
        '_peak_count',

        '_early_mean',
        '_mid_mean',
        '_mid2_mean',
        '_mid3_mean',
        '_late_mean',

        '_early_std',
        '_mid_std',
        '_mid2_std',
        '_mid3_std',
        '_late_std',

        '_early_late_diff',

        '_very_late_mean',
        '_very_late_std',
        '_very_late_max',
        '_very_late_min',


        '_late_intensity',
        '_late_peak_count',
        '_late_accel_mean',
        '_late_accel_max',
        '_late_jerk_mean',
        # f'{col}_late_jerk_max', # kötü

        '_final_velocity',
        '_deceleration_pattern',
        # f'{col}_completion_smoothness',
        '_late_energy',
        # f'{col}_termination_ratio', # kötü

]

In [ ]:
import uuid
import os
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from catboost import CatBoostClassifier
from datetime import datetime

# --- Assumptions: ---
# `train`               : pd.DataFrame, with your features + CONFIG.SUBJECT + CONFIG.TARGET
# `train_demographics`  : pd.DataFrame, with columns [CONFIG.SUBJECT, 'adult_child', 'handedness', 'sex']
# `imu_cols`            : List[str], your raw IMU column names
# `feature_suffixes_to_add`: List[str], as defined previously
# `CONFIG`              : object with attributes FOLDS (int), SUBJECT (str), TARGET (str), NUM_CLASSES (int)
# `cat_params`          : dict, your CatBoost training parameters
# `le`                  : fitted LabelEncoder on CONFIG.TARGET
# `metric_calculator`   : object with method calculate_hierarchical_f1(sol_df, sub_df)



# =============================================================================
# 0. Prepare run ID and output directory
run_id = uuid.uuid4()
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# =============================================================================
# 1. Precompute Multilabel Stratified K-Folds on demographics
t_d = train_demographics.copy()
skf = MultilabelStratifiedKFold(
    n_splits=CONFIG.FOLDS,
    shuffle=True,
    random_state=42
)

for fold, (tr_idx, val_idx) in enumerate(
    skf.split(
        t_d,
        t_d[['adult_child', 'handedness', 'sex']]
    )
):
    t_d.loc[val_idx, 'fold'] = fold

t_d['fold'] = t_d['fold'].astype(int)
print("Demographics fold distribution:\n", t_d['fold'].value_counts(), "\n")

# 2. Merge fold assignments into main train set
train = train.merge(
    t_d[[CONFIG.SUBJECT, 'fold']],
    on=CONFIG.SUBJECT,
    how='left'
)

# =============================================================================
# 3. Helper function: trains & evaluates across precomputed folds
def train_and_evaluate_fold(
    feature_list: list,
    train_data: pd.DataFrame,
    config,
    cat_params: dict,
    le,
    metric_calculator
):
    unique_features = sorted(set(feature_list), key=feature_list.index)
    oof_preds = np.zeros(len(train_data), dtype=int)

    for fold in range(config.FOLDS):
        # split by precomputed fold column
        train_idx = train_data.index[train_data['fold'] != fold].tolist()
        valid_idx = train_data.index[train_data['fold'] == fold].tolist()

        X_tr = train_data.loc[train_idx, unique_features].reset_index(drop=True)
        y_tr = train_data.loc[train_idx, config.TARGET].reset_index(drop=True)
        X_va = train_data.loc[valid_idx, unique_features].reset_index(drop=True)
        y_va = train_data.loc[valid_idx, config.TARGET].reset_index(drop=True)

        model = CatBoostClassifier(**cat_params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            use_best_model=True,
            verbose=0
        )

        # save each fold model (optional)
        model_path = os.path.join(
            models_dir,
            f"cat_fold_{fold+1}_{run_id}.pkl"
        )
        model.save_model(model_path)

        proba = model.predict_proba(X_va)
        oof_preds[valid_idx] = np.argmax(proba, axis=1)

    # invert labels and compute overall OOF score
    y_true_orig = le.inverse_transform(train_data[config.TARGET].to_numpy())
    y_pred_orig = le.inverse_transform(oof_preds)
    sol_df = pd.DataFrame({"gesture": y_true_orig})
    sub_df = pd.DataFrame({"gesture": y_pred_orig})
    overall_oof = metric_calculator.calculate_hierarchical_f1(sol_df, sub_df)

    return overall_oof, None, None

# =============================================================================
# 4. Forward Selection Loop
SCORE_IMPROVEMENT_THRESHOLD = 0.009

print("Starting Threshold-based Forward Feature Selection...")
print(f"Minimum required improvement: {SCORE_IMPROVEMENT_THRESHOLD}")
print("="*60)

# Baseline score with raw IMU features
current_features = imu_cols.copy()
baseline_oof, _, _ = train_and_evaluate_fold(
    current_features, train, CONFIG, cat_params, le, metric_calculator
)
print(f"Baseline OOF ({len(current_features)} features): {baseline_oof:.6f}")
previous_oof = baseline_oof

# Tracking
all_results = []
final_selected_features = current_features.copy()

# Iteratively test each suffix
for suffix in feature_suffixes_to_add:
    print(f"\n--- Testing feature suffix: '{suffix}' ---")
    if suffix in [
        'normalized_position',
        'late_acc_rot_coordination',
        'late_total_motion',
        'late_total_motion_std',
        'late_acc_acc_rot_ratio'
    ]:
        new_feats = [suffix]
    else:
        new_feats = [f"{col}{suffix}" for col in imu_cols]

    candidate_features = final_selected_features + new_feats
    candidate_features = [
        f for f in candidate_features
        if f in train.columns
    ]

    new_oof, _, _ = train_and_evaluate_fold(
        candidate_features, train, CONFIG, cat_params, le, metric_calculator
    )
    gain = new_oof - previous_oof
    added = len(candidate_features) - len(final_selected_features)

    print(
        f"Tested {len(candidate_features)} features → "
        f"Prev OOF: {previous_oof:.6f}, New OOF: {new_oof:.6f}, "
        f"Gain: {gain:+.6f}"
    )

    if gain > SCORE_IMPROVEMENT_THRESHOLD:
        decision = "KEPT"
        final_selected_features = candidate_features
        previous_oof = new_oof
        print(f"✔ KEEP '{suffix}' (gain {gain:.6f})")
    else:
        decision = "DISCARDED"
        print(f"✘ DISCARD '{suffix}' (gain {gain:.6f})")

    all_results.append({
        'suffix': suffix,
        'decision': decision,
        'gain': gain,
        'new_oof': new_oof,
        'features_added': added
    })

# =============================================================================
# 5. Summary & Save
results_df = pd.DataFrame(all_results)
kept = results_df[results_df.decision == "KEPT"]
discarded = results_df[results_df.decision == "DISCARDED"]

print("\n" + "="*60)
print("Forward Selection Complete")
print(f"Final OOF Score: {previous_oof:.6f}")
print(f"Total features selected: {len(final_selected_features)}\n")

print("Kept suffixes:")
print(kept[['suffix','gain']].to_string(index=False) if not kept.empty else "  None")

print("\nDiscarded suffixes:")
print(discarded[['suffix','gain']].to_string(index=False) if not discarded.empty else "  None")

# Save final feature list
with open('final_feature_list.txt', 'w') as f:
    for feat in final_selected_features:
        f.write(feat + "\n")
print("\nSaved final feature list to 'final_feature_list.txt'")


In [ ]:
# avg_acceleration


In [ ]:
feat_imp_df